# VRAM Profiling Demo

This notebook demonstrates GPU memory profiling capabilities for ML model optimization.

In [ ]:
# Import necessary libraries
import sys
sys.path.append('../projects/gpu_optimizer/src')
sys.path.append('../projects/shared_utils/src')

import torch
import psutil
import matplotlib.pyplot as plt
from gpu_optimizer import VRAMProfiler, ModelOptimizer
from shared_utils import load_config, setup_logging

In [ ]:
# Setup
config = load_config('../config/machines.yml')
logger = setup_logging('vram_profiling_demo')

# Initialize profiler
profiler = VRAMProfiler(config)
optimizer = ModelOptimizer(config)
print("VRAM profiler initialized successfully")

## GPU Memory Analysis

In [ ]:
# Check GPU availability and memory
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    total_memory = torch.cuda.get_device_properties(device).total_memory
    allocated_memory = torch.cuda.memory_allocated(device)
    cached_memory = torch.cuda.memory_reserved(device)
    
    print(f"GPU Device: {torch.cuda.get_device_name(device)}")
    print(f"Total Memory: {total_memory / 1024**3:.2f} GB")
    print(f"Allocated Memory: {allocated_memory / 1024**3:.2f} GB")
    print(f"Cached Memory: {cached_memory / 1024**3:.2f} GB")
else:
    print("No CUDA GPU available")

## Model Memory Profiling

In [ ]:
# Profile a sample model
def profile_model_memory():
    # Create a simple model for demonstration
    model = torch.nn.Sequential(
        torch.nn.Linear(784, 512),
        torch.nn.ReLU(),
        torch.nn.Linear(512, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 10)
    )
    
    if torch.cuda.is_available():
        model = model.cuda()
        
        # Profile memory usage
        memory_profile = profiler.profile_model(model)
        print(f"Model memory profile: {memory_profile}")
        
        return model, memory_profile
    else:
        print("Cannot profile model memory - no GPU available")
        return model, None

model, profile = profile_model_memory()

## Optimization Recommendations

In [ ]:
# Get optimization recommendations
if profile:
    recommendations = optimizer.get_optimization_recommendations(profile)
    print("Optimization Recommendations:")
    for rec in recommendations:
        print(f"- {rec}")